In [2]:
import zipfile
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

%matplotlib inline

In [3]:
import os
print(os.getcwd())

C:\Users\ccomk\jupyter_work\IBM Competition\Spot-Challenge-Wildfires\main


In [4]:
H_Wildfires = pd.read_csv(r'..\data\Nov_10\Historical_Wildfires.csv')
H_Weather = pd.read_csv(r'..\data\Nov_10\HistoricalWeather.csv')
H_Weeather_Fc = pd.read_csv(r'..\data\Nov_10\HistoricalWeatherForecasts.csv')
LandClass = pd.read_csv(r'..\data\Nov_10\LandClass.csv')
Vege = pd.read_csv(r'..\data\Nov_10\VegetationIndex.csv')

In [5]:
H_Wildfires.head()

,Region,Date,Estimated_fire_area,Mean_estimated_fire_brightness,Mean_estimated_fire_radiative_power,Mean_confidence,Std_confidence,Var_confidence,Count,Replaced
0,NSW,1/4/2005,8.68000,312.266667,42.400000,78.666667,2.886751,8.333333,3,R
1,NSW,1/5/2005,16.61125,322.475000,62.362500,85.500000,8.088793,65.428571,8,R
2,NSW,1/6/2005,5.52000,325.266667,38.400000,78.333333,3.214550,10.333333,3,R
3,NSW,1/7/2005,6.26400,313.870000,33.800000,92.200000,7.529940,56.700000,5,R
4,NSW,1/8/2005,5.40000,337.383333,122.533333,91.000000,7.937254,63.000000,3,R


In [28]:
H_Weather.head(8)


,Date,Region,Parameter,count()[unit: km^2],min(),max(),mean(),variance()
0,2005-01-01,NSW,Precipitation,8.002343e+05,0.000000,1.836935,0.044274,0.028362
1,2005-01-01,NSW,RelativeHumidity,8.002343e+05,13.877194,80.522964,36.355567,253.559937
2,2005-01-01,NSW,SoilWaterContent,8.002343e+05,0.002245,0.414305,0.170931,0.007758
3,2005-01-01,NSW,SolarRadiation,8.002343e+05,14.515009,32.169781,26.749389,6.078587
4,2005-01-01,NSW,Temperature,8.002343e+05,14.485785,35.878704,27.341182,18.562212
5,2005-01-01,NSW,WindSpeed,8.002343e+05,1.354448,7.670482,3.323550,0.850048
6,2005-01-01,NT,Precipitation,1.357561e+06,0.000000,315.266815,9.884958,546.059262
7,2005-01-01,NT,RelativeHumidity,1.357561e+06,14.558820,95.683342,61.494675,584.201131


In [32]:
Unit_info = {'Data':['Precipitation', 'Relative humidity', 'Soil water content', 'Solar radiation','Temperature','Wind speed'],
        'Unit':['mm/day','%','m^3','MJ/day','C','m/s']}
data_type = pd.DataFrame(Unit_info)
data_type

,Data,Unit
0,Precipitation,mm/day
1,Relative humidity,%
2,Soil water content,m^3
3,Solar radiation,MJ/day
4,Temperature,C
5,Wind speed,m/s


##### why no Precipitation & Soil water content here ?

In [34]:
H_Weeather_Fc.head(8)

,Date,Region,Parameter,Lead time,count()[unit: km^2],min(),max(),mean(),variance()
0,2014-01-01,NSW,RelativeHumidity,5,8.037682e+05,7.482927,85.021118,28.223569,353.620815
1,2014-01-01,NSW,SolarRadiation,5,8.037682e+05,24.865765,33.557598,31.647308,2.276068
2,2014-01-01,NSW,Temperature,5,8.037682e+05,21.243755,36.929035,30.893523,17.918553
3,2014-01-01,NSW,WindSpeed,5,8.037682e+05,1.593531,6.989559,3.958822,1.334834
4,2014-01-01,NT,RelativeHumidity,5,1.349817e+06,14.796251,73.601479,39.799856,189.805002
5,2014-01-01,NT,SolarRadiation,5,1.349817e+06,18.851019,32.766205,28.016282,6.573279
6,2014-01-01,NT,Temperature,5,1.349817e+06,25.955570,39.907539,33.379110,6.605778
7,2014-01-01,NT,WindSpeed,5,1.349817e+06,1.192912,12.156700,4.740253,4.871655


##### Every Unit is '%'

In [36]:
LandClass

,Region,Shrubs,Herbaceous vegetation,Cultivated and managed vegetation/agriculture (cropland),Urban / built up,Bare / sparse vegetation,Permanent water bodies,Herbaceous wetland,"Closed forest, evergreen, broad leaf","Closed forest, deciduous broad leaf","Closed forest, unknown","Open forest, evergreen broad leaf","Open forest, deciduous broad leaf","Open forest, unknown definitions",Open sea
0,NSW,6.2,43.6,13.0,0.3,0.2,0.2,0.1,14.7,6.8,0.3,0.5,3.7,10.3,0.1
1,NT,18.1,48.9,0.1,0.0,0.4,0.1,0.1,1.0,7.7,0.1,0.1,13.6,9.3,0.5
2,QL,9.5,45.3,1.6,0.1,1.1,0.1,0.0,5.3,13.3,0.3,0.1,12.0,11.0,0.3
3,SA,24.1,54.8,5.8,0.1,4.8,1.2,0.1,0.3,1.3,0.1,0.1,1.4,5.3,0.6
4,TA,0.7,23.8,1.2,0.2,0.1,1.9,1.4,50.1,0.6,1.1,7.2,1.5,7.2,3.0
5,WA,31.3,43.5,5.6,0.0,1.0,0.4,0.0,1.2,2.4,0.2,0.1,4.8,8.7,0.8
6,VI,1.4,35.0,23.3,1.0,0.1,0.6,0.3,23.9,3.8,0.3,1.5,2.7,5.2,0.9


#### The row is monthly basis

##### A Vegetation Index (VI) is a spectral transformation of two or more bands designed to enhance the contribution of vegetation properties and allow reliable spatial and temporal inter-comparisons of terrestrial photosynthetic activity and canopy structural variations

In [37]:
Vege.head(13)

,Region,Date,Vegetation_index_mean,Vegetation_index_max,Vegetation_index_min,Vegetation_index_std,Vegetation_index_variance
0,NSW,1/1/2005,0.349202,0.9972,-0.2,0.204862,0.041968
1,NSW,2/1/2005,0.357403,0.9772,-0.2,0.208673,0.043544
2,NSW,3/1/2005,0.354087,0.9750,-0.2,0.209450,0.043869
3,NSW,4/1/2005,0.347242,0.9904,-0.2,0.207307,0.042976
4,NSW,5/1/2005,0.345526,0.9972,-0.2,0.202858,0.041151
5,NSW,6/1/2005,0.399652,0.9602,-0.2,0.188570,0.035559
6,NSW,7/1/2005,0.451143,0.9555,-0.2,0.174267,0.030369
7,NSW,8/1/2005,0.498218,0.9782,-0.2,0.181103,0.032798
8,NSW,9/1/2005,0.505553,0.9805,-0.2,0.204202,0.041698
9,NSW,10/1/2005,0.439443,0.9814,-0.2,0.214671,0.046084
